##Project Context##

Here is the hugging face datasets we chose to use:

The large version is of size (>2 million) so we will not be able to train on the entire dataset.

https://huggingface.co/datasets/recipe_nlg

We looked for an resource that did something similar to recipe generation and fine-tuning GPT-2. We came across this article that fine-tuned a GPT-2 model to write songs and felt we could use their code as a jumping off point for our fine-tuning of GPT-2 for recipe generation. The code in this article was a skeleton for our work.


Trying to follow: https://towardsdatascience.com/how-to-fine-tune-gpt-2-for-text-generation-ae2ea53bc272




This is how gpt-2 performs without having been further trained on our recipe nlg dataset. It is pretty good but occasionally produces strange ingredients. It also doesnt always follow the same format or a clean format of writing up the recipe.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

#loading pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

#input a recipe name
input_recipe_name = "Chocolate Cake"
input_ids = tokenizer.encode(input_recipe_name, return_tensors="pt")

#generates a recipe
output1 = model.generate(input_ids, max_length=200, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
#output2 = model.generate(input_ids, max_length=200, num_return_sequences=1, temperature=.8)
#print
generated_recipe1 = tokenizer.decode(output1[0], skip_special_tokens=True)
#generated_recipe2 = tokenizer.decode(output2[0], skip_special_tokens=True)
print("Generated Recipe:", generated_recipe1)
#print("Generated Recipe:", generated_recipe2)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Recipe: Chocolate Cake

1/2 cup chocolate chips, melted (I used 1/4 cup, but you can use more if you like) 1 cup unsalted butter, at room temperature 1 tsp vanilla extract (optional) Instructions Preheat oven to 350 degrees F. Line a 9x13 baking sheet with parchment paper. In a large bowl, whisk together the melted butter and vanilla. Stir in the cocoa powder, salt, and pepper. Pour into the prepared baking dish. Bake for 20-25 minutes, or until a toothpick comes out clean. Remove from the oven and allow to cool completely before serving. Nutrition Facts Chocolate Cake Amount Per Serving (1 cup) Calories 683 Calories from Fat 9 % Daily Value* Total Fat 2g 2% Saturated Fat 1g 1% Cholesterol 0mg 0% Sodium 7mg 2 % Total Carbohydrates 6g 3% Dietary Fiber 2.5g 4% Sugars 1.4g Protein 1mg 1 % * Percent


"1/2 cup chocolate chips, melted (I used 1/4 cup, but you can use more if you like) 1 cup unsalted butter, at room temperature 1 tsp vanilla extract (optional) Instructions Preheat oven to 350 degrees F. Line a 9x13 baking sheet with parchment paper. In a large bowl, whisk together the melted butter and vanilla. Stir in the cocoa powder, salt, and pepper. Pour into the prepared baking dish. Bake for 20-25 minutes, or until a toothpick comes out clean. Remove from the oven and allow to cool completely before serving. Nutrition Facts Chocolate Cake Amount Per Serving (1 cup) Calories 683 Calories from Fat 9 % Daily Value* Total Fat 2g 2% Saturated Fat 1g 1% Cholesterol 0mg 0% Sodium 7mg 2 % Total Carbohydrates 6g 3% Dietary Fiber 2.5g 4% Sugars 1.4g Protein 1mg 1 % * Percent"

For example with this output, it is weird to cook a chocolate cake on a baking sheet which is typically used for cookies, it generated nutrition information which was not asked for, it included pepper as a chocolate cake ingredient. Also, there are no rising ingredients like baking powder or baking soda which is pretty important to a cake. Overall it did not follow the typical recipe delivery of listing ingredients one by one and then directions step by step.

**We hope to improve gpt-2 to have more structured and sensible outputs for recipes.**

## Data Preparation ##


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Use this cell for large dataset
##Had to download dataset to use the larger version; download here: https://recipenlg.cs.put.poznan.pl/dataset
##Uploaded to the shared project folder
from datasets import load_dataset


dataset_lg_path = '/content/drive/MyDrive/CS505_FinalProject/dataset/'

#maybe to improve memory space, change this to only load ingredients and directions...
dataset_large = load_dataset("recipe_nlg", data_dir=dataset_lg_path)
print(dataset_large)

Generating train split:   0%|          | 0/2231142 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'ingredients', 'directions', 'link', 'source', 'ner'],
        num_rows: 2231142
    })
})


In [ ]:
from tqdm import tqdm
#preparing inputs and outputs to train on

#list of recipe names
names = []

#list of recipe ingredients and directions
ing_dir = []

print(type(dataset_large['train']))
for example in tqdm(dataset_large['train']):
    if(len(names) >=10000):
        break

    recipe = "Ingredients: \n"
    for ingredient in example['ingredients']:
        recipe += f"{ingredient}\n"

    recipe += "\nDirections:\n"
    for i, step in enumerate(example['directions']):
        recipe += f"- {step}\n"

    names.append(example['title'])
    ing_dir.append(recipe)


max = 0
sum = 0
for item in ing_dir:
  if len(item) >max:
    max = len(item)
  sum += len(item)
print("max",max)
print("length of data", len(ing_dir))
print("avg length of data", sum/len(ing_dir))

import csv
# Specify the CSV file path
csv_file_path_ing_dir = '/content/drive/MyDrive/CS505_FinalProject/ing_dir.csv'

# Write data to the CSV file
with open(csv_file_path_ing_dir, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list to the CSV file
    writer.writerow(ing_dir)

print(f'Data saved to {csv_file_path_ing_dir}')


import csv
# Specify the CSV file path
csv_file_path_names = '/content/drive/MyDrive/CS505_FinalProject/names.csv'

# Write data to the CSV file
with open(csv_file_path_names, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the list to the CSV file
    writer.writerow(names)

print(f'Data saved to {csv_file_path_names}')



<class 'datasets.arrow_dataset.Dataset'>


  0%|          | 10000/2231142 [00:00<03:39, 10129.55it/s]


max 1669
length of data 10000
avg length of data 443.1478
Data saved to /content/drive/MyDrive/CS505_FinalProject/ing_dir.csv
Data saved to /content/drive/MyDrive/CS505_FinalProject/names.csv


In [ ]:
import csv
csv_file_path_ing_dir = '/content/drive/MyDrive/CS505_FinalProject/ing_dir.csv'
# Read data from the CSV file and convert it to a list
with open(csv_file_path_ing_dir, 'r') as file:
    reader = csv.reader(file)

    # Assuming the CSV file contains only one row
    outputs = [(item) for item in next(reader, [])]


In [ ]:

csv_file_path_names = '/content/drive/MyDrive/CS505_FinalProject/names.csv'
# Read data from the CSV file and convert it to a list
with open(csv_file_path_names, 'r') as file:
    reader = csv.reader(file)

    # Assuming the CSV file contains only one row
    inputs = [(item) for item in next(reader, [])]



Ideally, we want our recipe generations to be outputs to be very structured looking as follows:

In [ ]:
#size of our training dataset, # of recipies trained on
train_size = 6000
recipes_all = []
MAX_REC_LENGTH = 500
for i in range(len(inputs)):
  if(len(recipes_all)==train_size):
    break
  rec = "Recipe: " +inputs[i] + "\n" + outputs[i]
  if len(rec)<=MAX_REC_LENGTH:
    recipes_all.append(rec)

recipes = recipes_all[:5000]
recipes_test = recipes_all[5000:]

print(len(recipes))
for i in recipes[:5]:
  print(i)

print(len(recipes_test))
for i in recipes_test[:5]:
  print(i)

5000
Recipe: Jewell Ball'S Chicken
Ingredients: 
1 small jar chipped beef, cut up
4 boned chicken breasts
1 can cream of mushroom soup
1 carton sour cream

Directions:
- Place chipped beef on bottom of baking dish.
- Place chicken on top of beef.
- Mix soup and cream together; pour over chicken. Bake, uncovered, at 275° for 3 hours.

Recipe: Creamy Corn
Ingredients: 
2 (16 oz.) pkg. frozen corn
1 (8 oz.) pkg. cream cheese, cubed
1/3 c. butter, cubed
1/2 tsp. garlic powder
1/2 tsp. salt
1/4 tsp. pepper

Directions:
- In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings.

Recipe: Reeses Cups(Candy)  
Ingredients: 
1 c. peanut butter
3/4 c. graham cracker crumbs
1 c. melted butter
1 lb. (3 1/2 c.) powdered sugar
1 large pkg. chocolate chips

Directions:
- Combine first four ingredients and press in 13 x 9-inch ungreased pan.
- Melt chocolate chips and spread over mixture. Refr

##Custom Dataset##


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from tqdm import tqdm
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import Dataset

class CookBook(Dataset):
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=MAX_REC_LENGTH):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        pad_token_id = tokenizer.pad_token_id
        self.recipes = []

        for item in tqdm(recipes):
            encoded_sequence = self.tokenizer.encode(f"{item}", max_length=MAX_REC_LENGTH, truncation=True)
            # Check if the tokenizer has a specific padding token
            if pad_token_id is not None:
              # Apply padding if the sequence is shorter than MAX_REC_LENGTH
              padding_length = MAX_REC_LENGTH - len(encoded_sequence)
              encoded_sequence += [pad_token_id] * padding_length
            else:
              # Use a different method for padding (e.g., zero-padding)
              encoded_sequence += [0] * (max_length - len(encoded_sequence))
              #encoded_sequence += [tokenizer.pad_token_id] * padding_length
            if encoded_sequence == None:
              print(item)
            self.recipes.append(torch.tensor(encoded_sequence))
        if truncate:
            self.recipes = self.recipes[:max_length]
        self.recipes_count = len(self.recipes)

    def __len__(self):
        return self.recipes_count

    def __getitem__(self, item):
        return self.recipes[item]

dataset = CookBook(recipes, truncate=True, gpt2_type="gpt2")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

100%|██████████| 5000/5000 [00:03<00:00, 1356.95it/s]


##Training##

In [ ]:

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import torch

def train(
    dataset, model, tokenizer, lr=.001, epochs=25, batch_size=8, warmup_steps=200,
    output_dir="/content/drive/MyDrive/CS505_FinalProject/", output_prefix="wreckgar",
    save_model_on_epoch=False,
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in tqdm(range(epochs)):
        print(f"Training epoch {epoch}")

        for idx, input_tensor in enumerate(train_dataloader):
            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs.loss

            loss.backward()

            if (idx + 1) % batch_size == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )

    return model


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
pad_token_id = tokenizer.pad_token_id
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = train(dataset, model, tokenizer, lr=0.001)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/25 [00:00<?, ?it/s]

Training epoch 0


  4%|▍         | 1/25 [01:09<27:55, 69.83s/it]

Training epoch 1


  8%|▊         | 2/25 [02:18<26:35, 69.36s/it]

...

 92%|█████████▏| 23/25 [26:38<02:18, 69.43s/it]

Training epoch 23


 96%|█████████▌| 24/25 [27:47<01:09, 69.37s/it]

Training epoch 24


100%|██████████| 25/25 [28:56<00:00, 69.48s/it]


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
fine_tuned_model = model

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

# Set the model to evaluation mode
fine_tuned_model.eval()

# Prompt for generation
prompt = "Recipe: Chocolate Cake\n"


# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
#input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=False, truncation=True, pad_token_id=0).to(device)

# Generate text
output = fine_tuned_model.generate(input_ids, max_length=300, num_return_sequences=1, temperature=.8)
#output = fine_tuned_model.generate(input_ids, max_length=500, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Recipe: Chocolate Cake
Ingredients: 
1 1/2 c. flour
1/2 c. sugar
1/2 tsp. salt
2 eggs
1 c. buttermilk
1 Tbsp. cornstarch
1 tsp. soda
1 tsp. vanilla

Directions:
- Mix flour, sugar, salt and soda.
- Add eggs, cornstarch and vanilla.
- Pour into well greased and floured loaf pan.
- Bake in well floured loaf pan.
- Cool thoroughly before cutting.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


##Analysis of Model Performance##

Human Evaluation:

Testing to see how the model completes recipes it was not trained on. Ideally we want to see that it is finishing out the recipes physical structure, is not copying the training exactly (overfitting), and to make sure that its not spitting out giberish (underfitting).

Here we check the model on 3 recipes to get an idea of how it is performing. When this cell prints the "generated recipe" we signifiy what the model was prompted with by printing it in ALL CAPS. Generated text will follow in lower case.

Testing on unseen recipes:

Here we prompt the model with recipe name only. We want to see how the model writes directions given just the name.

In [ ]:
for recipe in recipes_test[:3]:
  print("Original Recipe")
  print(recipe)
  # Prompt for generation
  prompt_len = recipe.find("\n") + 1
  prompt = recipe[:prompt_len]
  #print("PROMPT:", prompt)

  # Tokenize the prompt
  input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
  #Generate text
  output = fine_tuned_model.generate(input_ids, max_length=300, num_return_sequences=1, temperature=.8)
  # Decode and print the generated text
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  print()
  print("Generated Recipe")
  print(generated_text[:prompt_len].upper() + generated_text[prompt_len:])
  print()
  print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Recipe
Recipe: Rice Salad
Ingredients: 
2 medium size carrots
4 stalks celery
1 1/2 c. cooked rice, cooled
1 c. drained, crushed pineapple
1/4 c. sugar
2 Tbsp. lemon juice

Directions:
- Cut up carrots and celery.
- Mix all ingredients well.
- Cover and chill before serving.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Recipe
RECIPE: RICE SALAD
Ingredients: 
2 cans or jars of celery soup
1 can cream of celery soup
1/2 can cream of mushroom soup
1/2 can cream of chicken soup
1 can cream of celery soup
1 1/2 c. milk
1/4 c. vinegar

Directions:
- Mix all ingredients together.
- Let sit overnight.
- Shape into a ball.
- Roll out, using only a little flour.
- For each roll, add 1 tablespoon salt and pepper.
- Mix well.
- Keep in refrigerator.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Original Recipe
Recipe: Eggplant And Tomato Casserole
Ingredients: 
1 eggplant, peeled and sliced
4 to 5 ripe tomatoes, peeled and sliced
grated swiss cheese
salt and pepper
bread crumbs
sweet basil

Directions:
- Soak eggplant slices in cold water for several hours.
- Drain and arrange layers of eggplant and tomatoes in a buttered baking dish.
- Sprinkle each layer with grated chees

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Recipe
RECIPE: EGGPLANT AND TOMATO CASSEROLE
Ingredients: 
1 can tomato soup
1/2 c. dry bread crumbs
1 (8 oz.) pkg. cream cheese
1/3 c. lemon juice
3/4 c. mayonnaise
1/3 c. chopped green onions
1/3 c. cottage cheese

Directions:
- Mix all ingredients together in baking dish.
- Cover and bake at 350° for 45 minutes.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Original Recipe
Recipe: Jane'S Coleslaw
Ingredients: 
4 c. chopped cabbage
8 green onions, chopped
4 Tbsp. sesame seed plus 4 Tbsp. almonds (all toasted)
1 c. sliced onions
2 pkg. Ramen noodles (chicken flavor)

Directions:
- Toast sesame seed and almonds in 2 tablespoons of butter until light brown.
- Cool.
- Break up noodles.
- Add onions, sesame seed and almonds.
- Toss with dressing.


Generated Recipe
RECIPE: JANE'S COLESLAW
Ingredients: 
1 c. sugar
2 c. vegeta

Analysis:

When only prompted with the title of a recipe, the model has some difficulty printing sensible recipes but follows intended structure strongly. Each line printed makes sense on its own.

Success:
 - It follows our intended structure of recipe output very well.
 - Sensible measurements are listed out (i.e. not a cup of salt or overserving of other potent ingredients)
 - Sometimes mentions ingredients in title
 - Most ingredients are mentioned in the directions


Room for improvement:
 - The model will list repetitve ingredients
 - Sometimes leaves out ingredients explicity mentioned in the title
 - Ingredients sometimes listed but not addressed in directions, and sometimes vice versa
 - Needs more understanding of a savory dish from sweet dish


Here we prompt the model with recipe name and ingredients, everything up until directions. We want to see how the model writes directions given ingredients and name. We anticipate its performance will be better with the additional context.

In [ ]:
for recipe in recipes_test[:3]:
  print("Original Recipe")
  print(recipe)
  # Prompt for generation
  prompt_len = recipe.find("Directions") + len("Directions")

  prompt = recipe[:prompt_len]

  # Tokenize the prompt
  input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
  #Generate text
  output = fine_tuned_model.generate(input_ids, max_length=300, num_return_sequences=1, temperature=.8)
  # Decode and print the generated text
  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
  print()
  print("Generated Recipe")
  print(generated_text[:prompt_len].upper() + generated_text[prompt_len:])
  print()
  print()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Original Recipe
Recipe: Rice Salad
Ingredients: 
2 medium size carrots
4 stalks celery
1 1/2 c. cooked rice, cooled
1 c. drained, crushed pineapple
1/4 c. sugar
2 Tbsp. lemon juice

Directions:
- Cut up carrots and celery.
- Mix all ingredients well.
- Cover and chill before serving.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Recipe
RECIPE: RICE SALAD
INGREDIENTS: 
2 MEDIUM SIZE CARROTS
4 STALKS CELERY
1 1/2 C. COOKED RICE, COOLED
1 C. DRAINED, CRUSHED PINEAPPLE
1/4 C. SUGAR
2 TBSP. LEMON JUICE

DIRECTIONS:
- Sprinkle vegetables with sugar and lemon juice.
- Bring to a boil and boil for 2 minutes.
- Remove from stove and add vegetables.
- Stir well.
- Garnish with crushed pineapple.
- Store in refrigerator.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Original Recipe
Recipe: Eggplant And Tomato Casserole
Ingredients: 
1 eggplant, peeled and sliced
4 to 5 ripe tomatoes, peeled and sliced
grated swiss cheese
salt and pepper
bread crumbs
sweet basil

Directions:
- Soak eggplant slices in cold water for several hours.
- Drain and arrange layers of eggplant and tomatoes in a buttered baking dish.
- Sprinkle each layer with grated cheese, salt and pepper. 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated Recipe
RECIPE: EGGPLANT AND TOMATO CASSEROLE
INGREDIENTS: 
1 EGGPLANT, PEELED AND SLICED
4 TO 5 RIPE TOMATOES, PEELED AND SLICED
GRATED SWISS CHEESE
SALT AND PEPPER
BREAD CRUMBS
SWEET BASIL

DIRECTIONS:
- Heat oven to 425°.
- Peel and slice all the eggplant and put in bottom of pan.
- Melt butter and pour over mixture.
- Bake 40 minutes.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Original Recipe
Recipe: Jane'S Coleslaw
Ingredients: 
4 c. chopped cabbage
8 green onions, chopped
4 Tbsp. sesame seed plus 4 Tbsp. almonds (all toasted)
1 c. sliced onions
2 pkg. Ramen noodles (chicken flavor)

Directions:
- Toast sesame seed and almonds in 2 tablespoons of butter until light brown.
- Cool.
- Break up noodles.
- Add onions, sesame seed and almonds.
- Toss with dressing.


Generated Recipe
RECIPE: JANE'S COLESLAW
INGRED

Analysis:

When only prompted with the title of a recipe and ingredients, the model has a easier time generating sensible recipes. This is likely due to the added context of ingredients which provide much more info to the GPT then just a recipe name as a prompt.

Success:
 - It follows our intended structure of recipe output very well.
 - Majority of ingredients listed are mentioned in the directions (unlikely to bring up a new ingreedient in directions)


Room for improvement:
 - Sometimes does not mention a ingredient listed in prompt in the directions
 - Needs a better understanding of cooking methods (i.e. what to bake or boil, how hot and how long, when no heat is required)

##Conclusion##

It is pretty evident from human analysis of our outputs that with such a small training set (due to colab limits) our model has difficulty extracting meaning from recipe titles alone. Typically, these are creative and subjective to the recipe writer. It would be difficult for the model to figure out that Jane's Coleslaw is very simliar to John and everyone else's coleslaw on such a small sample size.

However, we were able to see that our model was able to learn from something more standardized and concrete to each recipe, the ingredients. If trained and fine tuned on a larger dataset this could be really applicable to home usage. Say you have a list of ingredients that you are limited to and would like the model to output a new recipe for those ingredients, this model could be very useful and inventive in that case.

While are model still has some improvements as listed above, these are promising initial results for a much larger scale recipe generating model.